In [1]:
'''
# Code Explainer

In this lecture we are going to build a code explainer that will create a docstring of a given function! 
'''

import inspect  # Used to transform python code to a string
import os
import openai

In [2]:
def hello(name):
    print(f'Hello {name}')

In [3]:
print(inspect.getsource(hello))

def hello(name):
    print(f'Hello {name}')



In [5]:
def docstring_prompt(code):
    prompt = f"{code}\n # A high quality python docstring of the above python function:\n\"\"\""
    return prompt

In [6]:
print(docstring_prompt(inspect.getsource(hello)))

def hello(name):
    print(f'Hello {name}')

 # A high quality python docstring of the above python function:
"""


In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=docstring_prompt(inspect.getsource(hello)),
  temperature=0,
  max_tokens=64,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\"\"\""]

)

In [9]:
print(response["choices"][0]["text"])


This function prints a greeting to the given name.

Parameters:
    name (str): The name of the person to greet.

Returns:
    None



In [10]:
## Combine docstring and function
def merge_docstring_and_function(original_function, docstring):
    function_string = inspect.getsource(original_function)
    split = function_string.split("\n")
    first_part, second_part = split[0], split[1:]
    # note the spaces before the three "
    merged_function = first_part + "\n" + '    """' + docstring + '    """' + "\n" + "\n".join(second_part)
    return merged_function

In [11]:
merged_code = merge_docstring_and_function(hello, response["choices"][0]["text"])

In [12]:
print(merged_code)

def hello(name):
    """
This function prints a greeting to the given name.

Parameters:
    name (str): The name of the person to greet.

Returns:
    None
    """
    print(f'Hello {name}')



In [13]:
## Get the docstring of a more complicated function
def test_function(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view

In [15]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=docstring_prompt(inspect.getsource(test_function)),
  temperature=0,
  max_tokens=256,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\"\"\""] # Corresponds to """, the end of the docstring

)

In [16]:
merged_code = merge_docstring_and_function(test_function, response["choices"][0]["text"])
print(merged_code)

def test_function(test, num_questions):
    """
This function takes in a test string and the number of questions in the test as parameters and returns a dictionary with the questions as keys and the corresponding question text as values.

Parameters:
test (str): The test string containing the questions and answers.
num_questions (int): The number of questions in the test.

Returns:
student_view (dict): A dictionary with the questions as keys and the corresponding question text as values.

    """
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view



In [17]:
def take(student_view):
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers

In [18]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=docstring_prompt(inspect.getsource(take)),
  temperature=0,
  max_tokens=256,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\"\"\""] # Corresponds to """, the end of the docstring

)

In [20]:
merged_code = merge_docstring_and_function(take, response["choices"][0]["text"])
print(merged_code)

def take(student_view):
    """
This function takes a student view as an argument and prints out the questions associated with it. It then takes the user's input and stores it in a dictionary with the question as the key and the answer as the value.

Parameters:
    student_view (dict): A dictionary containing questions as keys and their associated views as values.

Returns:
    answers (dict): A dictionary containing questions as keys and the user's answers as values.
    """
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers

